The aim of this predictive model is to predict if a certain person will receive a bank loan or not, using machine learning and deep learning tools.

In [59]:
# importations sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, cross_val_score, validation_curve, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [60]:
# importations pytorch
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader                    # to build data set with pytorch
import torch.nn as nn 
from torch.nn import Module                                                             # neural network
import torch.optim as optim
from torch.optim import Optimizer
from torch.nn.modules.loss import _Loss
import torchmetrics
from torchmetrics import Metric


In [61]:
dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")


KeyboardInterrupt: 

In [ ]:
head=dataset.head()
info=dataset.info()
print(head)
print(info)                                                                         # 28632 rows and 78 columns, there are 3 Dtypes ( float, int and object=[string] )
target_name= "action_taken_name"                                                    # purpose name
data , target = dataset.drop(columns=target_name), dataset[target_name]             # data to browse and target

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28632 entries, 0 to 28631
Data columns (total 78 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   as_of_year                      28632 non-null  int64  
 1   respondent_id                   28632 non-null  object 
 2   agency_name                     28632 non-null  object 
 3   agency_abbr                     28632 non-null  object 
 4   agency_code                     28632 non-null  int64  
 5   loan_type_name                  28632 non-null  object 
 6   loan_type                       28632 non-null  int64  
 7   property_type_name              28632 non-null  object 
 8   property_type                   28632 non-null  int64  
 9   loan_purpose_name               28632 non-null  object 
 10  loan_purpose                    28632 non-null  int64  
 11  owner_occupancy_name            28632 non-null  object 
 12  owner_occupancy                 

In [ ]:
# valeurs numeriques
dataset.select_dtypes(include='number')                      # there are 45 numercal features

,as_of_year,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,msamd,...,edit_status_name,edit_status,sequence_number,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,application_date_indicator
0,2017,9,3,1,1,1,359,3,1,11260.0,...,NaN,NaN,NaN,6702.0,17.170000,89400.0,94.010002,1651.0,2219.0,NaN
1,2017,5,1,1,1,1,99,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,9,2,1,1,1,247,3,1,21820.0,...,NaN,NaN,NaN,11450.0,20.030001,84800.0,99.690002,2923.0,3992.0,NaN
3,2017,5,1,1,1,1,292,3,1,11260.0,...,NaN,NaN,NaN,3864.0,22.490000,89400.0,71.989998,979.0,1328.0,NaN
4,2017,3,1,1,1,1,589,3,1,11260.0,...,NaN,NaN,NaN,1941.0,56.880001,89400.0,58.340000,179.0,318.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28627,2017,7,1,1,3,1,341,3,1,11260.0,...,NaN,NaN,NaN,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0,NaN
28628,2017,7,1,1,3,2,80,3,3,11260.0,...,NaN,NaN,NaN,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0,NaN
28629,2017,7,3,1,3,1,199,3,1,NaN,...,NaN,NaN,NaN,4619.0,71.339996,78400.0,130.669998,213.0,680.0,NaN
28630,2017,9,1,1,1,1,280,3,1,NaN,...,NaN,NaN,NaN,4619.0,71.339996,78400.0,130.669998,213.0,680.0,NaN


In [ ]:
#==Data qualities

dataset.isnull().sum()                                                                              # number of missing values 

# remove columns that has more than 30% of empty values

pourcentage_manquant=dataset.isnull().mean()*100                                                                                               

colonnes_a_supprimer=pourcentage_manquant[pourcentage_manquant>30].index.tolist()                   # remove

data_clean=data.drop(columns=colonnes_a_supprimer)                                                  # new datas


# doublons
data_clean.duplicated().sum()                                                                       # number of duplicates per lines

data_clean.drop_duplicates()                                                                        # removes duplicates




,as_of_year,respondent_id,agency_name,agency_abbr,agency_code,loan_type_name,loan_type,property_type_name,property_type,loan_purpose_name,...,hoepa_status_name,hoepa_status,lien_status_name,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units
0,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,VA-guaranteed,3,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,6702.0,17.170000,89400.0,94.010002,1651.0,2219.0
1,2017,0000005892,National Credit Union Administration,NCUA,5,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,FHA-insured,2,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,11450.0,20.030001,84800.0,99.690002,2923.0,3992.0
3,2017,0000005912,National Credit Union Administration,NCUA,5,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,3864.0,22.490000,89400.0,71.989998,979.0,1328.0
4,2017,92-0164378,Federal Deposit Insurance Corporation,FDIC,3,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,1941.0,56.880001,89400.0,58.340000,179.0,318.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28627,2017,7197000003,Department of Housing and Urban Development,HUD,7,Conventional,1,One-to-four family dwelling (other than manufa...,1,Refinancing,...,Not a HOEPA loan,2,Secured by a first lien,1,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0
28628,2017,04-3660901,Department of Housing and Urban Development,HUD,7,Conventional,1,One-to-four family dwelling (other than manufa...,1,Refinancing,...,Not a HOEPA loan,2,Secured by a first lien,1,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0
28629,2017,7197000003,Department of Housing and Urban Development,HUD,7,VA-guaranteed,3,One-to-four family dwelling (other than manufa...,1,Refinancing,...,Not a HOEPA loan,2,Secured by a first lien,1,4619.0,71.339996,78400.0,130.669998,213.0,680.0
28630,2017,0000280110,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,4619.0,71.339996,78400.0,130.669998,213.0,680.0


In [ ]:
# verification of data quality
col="as_of_year"                                # not useful
print(data_clean[col].unique())                 # print all the different values available
print(data_clean[col].nunique())                # print all the different values of the column

data_clean=data_clean.drop(columns=col)         # deletes

# delete unuseful columns
data_clean=data_clean.drop(columns=["agency_abbr","agency_code","state_name", "state_abbr", "state_code"])
data_clean=data_clean.drop(columns="preapproval")

# deletes all the columns numerical( not pertinent) made from a categorical( eg: rad_name , rad, rad represents the fixed number for rad_name)

# Identifies all the columns which ends with "_name"
name_cols = [col for col in data_clean.columns if col.endswith("_name")]

# Extract the racine
radicals = [col[:-5] for col in name_cols]   # enlève "_name"

# deletes the columns which reprsents those radicals
cols_to_drop = [col for col in data_clean.columns if col in radicals]

# keeps the others
columns_to_keep = [col for col in data_clean.columns if col not in cols_to_drop]

data_clean=data_clean.drop(columns=cols_to_drop)

# deletes the lines which does not have any feature
data_clean.dropna(how="all")

[2017]
1


,respondent_id,agency_name,loan_type_name,property_type_name,loan_purpose_name,owner_occupancy_name,loan_amount_000s,preapproval_name,action_taken,msamd_name,...,applicant_income_000s,purchaser_type_name,hoepa_status_name,lien_status_name,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units
0,0000451965,Consumer Financial Protection Bureau,VA-guaranteed,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,359,Not applicable,1,Anchorage - AK,...,137.0,Ginnie Mae (GNMA),Not a HOEPA loan,Secured by a first lien,6702.0,17.170000,89400.0,94.010002,1651.0,2219.0
1,0000005892,National Credit Union Administration,Conventional,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,99,Preapproval was not requested,1,NaN,...,45.0,"Life insurance company, credit union, mortgage...",Not a HOEPA loan,Secured by a first lien,NaN,NaN,NaN,NaN,NaN,NaN
2,0000451965,Consumer Financial Protection Bureau,FHA-insured,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,247,Not applicable,1,Fairbanks - AK,...,64.0,Ginnie Mae (GNMA),Not a HOEPA loan,Secured by a first lien,11450.0,20.030001,84800.0,99.690002,2923.0,3992.0
3,0000005912,National Credit Union Administration,Conventional,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,292,Not applicable,1,Anchorage - AK,...,120.0,"Commercial bank, savings bank or savings assoc...",Not a HOEPA loan,Secured by a first lien,3864.0,22.490000,89400.0,71.989998,979.0,1328.0
4,92-0164378,Federal Deposit Insurance Corporation,Conventional,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,589,Not applicable,1,Anchorage - AK,...,125.0,"Life insurance company, credit union, mortgage...",Not a HOEPA loan,Secured by a first lien,1941.0,56.880001,89400.0,58.340000,179.0,318.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28627,7197000003,Department of Housing and Urban Development,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Owner-occupied as a principal dwelling,341,Not applicable,1,Anchorage - AK,...,72.0,Freddie Mac (FHLMC),Not a HOEPA loan,Secured by a first lien,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0
28628,04-3660901,Department of Housing and Urban Development,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Not owner-occupied as a principal dwelling,80,Not applicable,3,Anchorage - AK,...,160.0,Loan was not originated or was not sold in cal...,Not a HOEPA loan,Secured by a first lien,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0
28629,7197000003,Department of Housing and Urban Development,VA-guaranteed,One-to-four family dwelling (other than manufa...,Refinancing,Owner-occupied as a principal dwelling,199,Not applicable,1,NaN,...,64.0,Ginnie Mae (GNMA),Not a HOEPA loan,Secured by a first lien,4619.0,71.339996,78400.0,130.669998,213.0,680.0
28630,0000280110,Consumer Financial Protection Bureau,Conventional,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,280,Not applicable,1,NaN,...,138.0,Fannie Mae (FNMA),Not a HOEPA loan,Secured by a first lien,4619.0,71.339996,78400.0,130.669998,213.0,680.0


In [ ]:
data_clean.info()                                                                                    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28632 entries, 0 to 28631
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   respondent_id                   28632 non-null  object 
 1   agency_name                     28632 non-null  object 
 2   loan_type_name                  28632 non-null  object 
 3   property_type_name              28632 non-null  object 
 4   loan_purpose_name               28632 non-null  object 
 5   owner_occupancy_name            28632 non-null  object 
 6   loan_amount_000s                28632 non-null  int64  
 7   preapproval_name                28632 non-null  object 
 8   action_taken                    28632 non-null  int64  
 9   msamd_name                      22213 non-null  object 
 10  county_name                     28514 non-null  object 
 11  county_code                     28523 non-null  float64
 12  census_tract_number             

In [ ]:
# classification of the data types
numerical_features=data_clean.select_dtypes(include='number').columns.tolist()                     
categorical_features=list(set(data_clean.columns)-set(numerical_features))                         
print(len(numerical_features))
print(len(categorical_features))

13
18


In [ ]:
# completion of absent data or nan
data_clean[numerical_features] = data_clean[numerical_features].fillna(data_clean[numerical_features].mean())                         # replace the absent value by the mean of the column

data_clean[categorical_features] = data_clean[categorical_features].apply(lambda col: col.fillna(col.mode()[0]))                      # replace the absent value by the mode


In [ ]:
# implementation of hgbc model
categorical_features= list(set(data_clean.columns)-set(numerical_features))

numeric= StandardScaler()
categoric = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric",numeric, numerical_features),
        ("categoric",categoric, categorical_features)
    ]
)
# model pipeloine and scaling complete
model_pipeline_full=Pipeline([ 
    ('preprocessor', preprocessor),
    ('rfc', RandomForestClassifier())
])

# data preparation
X_full=data_clean[numerical_features+categorical_features]
Y= target

# training
model_pipeline_full.fit(X_full,Y)

# predictive model
y_pred=model_pipeline_full.predict(X_full)

# evaluation of the model using cross validation

cv_results= cross_validate(
    model_pipeline_full,
    X_full,
    Y,
    cv=StratifiedKFold(n_splits=10),
    scoring= 'accuracy',
    return_train_score= True,
    return_estimator= True
)
# print the accuracy

mean_accuracy_full = cv_results['test_score'].mean()
print(f"Accuracy_full  (10-fold CV) : {mean_accuracy_full:.2f}")

Accuracy_full  (10-fold CV) : 0.97


In [ ]:
# implementation of the adc model

categorical_features= list(set(data_clean.columns)-set(numerical_features))

# Let's define a predictive model
numeric= StandardScaler()
categoric = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric",numeric, numerical_features),
        ("categoric",categoric, categorical_features)
    ]
)
# model pipeloine and scaling complete
model_pipeline_full=Pipeline([ 
    ('preprocessor', preprocessor),
    ('adc', AdaBoostClassifier())
])

# data preparation
X_full=data_clean[numerical_features+categorical_features]
Y= target

# training
model_pipeline_full.fit(X_full,Y)

# predictive model
y_pred=model_pipeline_full.predict(X_full)

# evaluation of the model using cross validation

cv_results= cross_validate(
    model_pipeline_full,
    X_full,
    Y,
    cv=StratifiedKFold(n_splits=10),
    scoring= 'accuracy',
    return_train_score= True,
    return_estimator= True
)
# print the accuracy

mean_accuracy_full = cv_results['test_score'].mean()
print(f"Accuracy_full  (10-fold CV) : {mean_accuracy_full:.2f}")

Accuracy_full  (10-fold CV) : 0.96


In [ ]:
print(pd.Series(Y).value_counts())   # occurrences


action_taken_name
Loan originated                                14430
Loan purchased by the institution               6415
Application withdrawn by applicant              3495
Application denied by financial institution     3075
File closed for incompleteness                   756
Application approved but not accepted            461
Name: count, dtype: int64


Let's Build and train a classification MLP on our data  with pytorch tools.

Step 1: Load the dataset using sklearn and create a custom PyTorch Dataset for this data.

In [ ]:
# implementation of the MLP

encoder=LabelEncoder()                                                                  # instance encoder
target_encoded=encoder.fit_transform(target)                                            # target encoding for tensor management

X_prepared=preprocessor.fit_transform(X_full)                                           # prepares X_full for pytorch( in numpy)

X=torch.tensor(X_prepared,dtype=torch.float32)                                          # data tensor building

means = X.mean(dim=0, keepdim=True)                                                     # mean
stds = X.std(dim=0, keepdim=True)                                                       # standard deviation(sigma)
 
stds[stds == 0] = 1.0                                                                   # avoid the 0-division 

X_standardized= (X - means) / stds                                                      # scaling

y=torch.tensor(target_encoded, dtype=torch.long)                                        # target tensor building                                           

data_set_torch=TensorDataset(X_standardized,y)                                          # data set building 

sample0, target0 = data_set_torch[0]
print (sample0.shape, target0.shape)                                                    # on obtient des vecteurs de dimensiosn 328 et le tzrget etant scalaore 

torch.Size([328]) torch.Size([])



Step 2: Create data loaders for training, validation, and testing.

In [ ]:
torch.manual_seed(42)

# define sizes for each step of the learning
train_size=len(data_set_torch)*80//100
valid_size=len(data_set_torch)*20//100
test_size=len(data_set_torch) - (train_size + valid_size)
train_dataset, valid_dataset , test_dataset = random_split(           
                                data_set_torch,
                                [train_size,valid_size,test_size]
                                ) 
# data loaders (to load data)
batch_size=32                                       # because it's a cpu
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
valid_loader=DataLoader(valid_dataset,batch_size=batch_size)
test_loader=DataLoader(test_dataset,batch_size=batch_size)


Step 3: Let's build a custom MLP module to tackle our classification task

In [ ]:
# let's start by the sequential way (most obvious)
X_tensor, y_tensor=data_set_torch.tensors
n_inputs= X_tensor.shape[1]
n_classes= len(torch.unique(y_tensor))                      # number of differents categories to predict  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# set the model
bank_loan_model = nn.Sequential(                       
    nn.Linear(n_inputs,200), nn.ReLU(),                    # linear layer , ReLU activation      
    nn.Linear(200,100), nn.ReLU(),
    nn.Linear(100,50), nn.ReLU(),
    nn.Linear(50, n_classes)
    ).to(device)                                           # puts on the cpu


In [62]:
     # let's build our training
n_epochs = 30
learning_rate= [0.01, 0.001 , 0.0001]

def train(
        
        model:Module,
        optimizer: Optimizer,
        criterion: _Loss ,                                                                            # criteria
        metric: Metric,                                                                               # metric
        train_loader: DataLoader , 
        valid_loader: DataLoader ,
        n_epochs:int,
        device: str = "cpu"
       
        )  :
    
        history={ "train_loss":[] , "train_metric":[], "valid_metric": []    }                             # initilization of the output

    

        for epoch in range(n_epochs):
            
            model.train()                                                                                  # sets the model in a training mode     
            train_loss=0.0                                                                                 # initialization
            metric.reset()

            for X_batch , y_batch in train_loader :
                X_batch, y_batch = X_batch.to(device),y_batch.to(device)                                   # sets on the device 
                y_pred=model(X_batch)
                loss = criterion(y_pred,y_batch)
                loss.backward()                                                                            # back propagation
                optimizer.step()                                                                           # updates model parameters
                optimizer.zero_grad()                                                                      # renitialize gradient to zero to avoid accumulation
                train_loss+= loss.item()       
                metric.update(y_pred,y_batch)                                                              # performance metric       

            train_loss/=len(train_loader)
            train_metric=metric.compute().item()

            # validation step

            model.eval()                                                                                     # set the model in a validation mode
            metric.reset()
            with torch.no_grad():                                                                            # avoid to compute gradient
                for X_val, y_val in valid_loader:
                    X_val, y_val = X_val.to(device), y_val.to(device)
                    outputs = model(X_val)
                    metric.update(outputs, y_val)

            valid_metric = metric.compute().item()


            history["train_loss"].append(train_loss)
            history["train_metric"].append(train_metric)
            history["valid_metric"].append(valid_metric)

        return history



# let's train our model
for lr in learning_rate:

    optimizer= torch.optim.SGD(bank_loan_model.parameters(), lr=lr)                            # optimizer with stochastic gradient descent
    criterion= nn.CrossEntropyLoss ()       
    metric= torchmetrics.Accuracy(task="multiclass",num_classes=n_classes).to(device)    

# out put

train(bank_loan_model, optimizer, criterion, metric, train_loader,                          # return the output of the training
                    valid_loader, n_epochs)




{'train_loss': [1.5830110159999164,
  1.568975829545346,
  1.5554529914642845,
  1.5423880477524337,
  1.5297710277847738,
  1.517721948676935,
  1.5061502469984513,
  1.4951849058686688,
  1.4847233308760148,
  1.474886003819258,
  1.4655052115464344,
  1.4567338955801958,
  1.448360855685932,
  1.4405018164458887,
  1.4330292816934638,
  1.425945923981054,
  1.419202701173015,
  1.412840684032973,
  1.4066630357137606,
  1.400692920112077,
  1.3949953678932936,
  1.3893879816851802,
  1.3839588305137678,
  1.3786188058513502,
  1.3732390625183808,
  1.3681517001137387,
  1.3628715991807383,
  1.3576279024505082,
  1.3524676423998518,
  1.347203447082855],
 'train_metric': [0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.5036018490791321,
  0.503601

In [ ]:
# test step with test_loader

def evaluate(
    model: Module,
    criterion: _Loss,
    test_loader: DataLoader,
    device: str = "cpu",
    aggregate_fn=torch.mean

    ):

    model.eval()          
    metrics=[]    

    with torch.no_grad(): 
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            metrics.append(loss)
            

            
    return aggregate_fn(torch.stack(metrics))


evaluate(bank_loan_model,criterion=nn.CrossEntropyLoss(), test_loader=test_loader )

tensor(1.3549)